In [ ]:
import logging.config
import os
import pathlib

Set visible gpu.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1, 3"

Configure logging

In [ ]:
output_dir = pathlib.Path('/data/wzy/WifiEnvironment/hparam_cmp')
try:
    os.makedirs(output_dir)
except FileExistsError:
    if not os.path.isdir(output_dir):
        raise FileExistsError("Please provide a path to a non-existing or directory as the output directory.")
log_format = '%(asctime)s: %(levelname)s: %(name)s: %(filename)s: %(funcName)s(): %(lineno)d:\t%(message)s'
formatter = logging.Formatter(log_format)
file_handler = logging.FileHandler(str(output_dir / 'log.log'), mode = 'a')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
formatter = logging.Formatter(fmt="[%(levelname)s %(asctime)s] %(name)s: %(message)s", datefmt="%m-%d %H:%M:%S",)
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
stream_handler.setFormatter(formatter)
logging.basicConfig(level = logging.DEBUG, handlers = [file_handler, stream_handler])

In [ ]:
import ray
import ray.tune as tune

import configs2
import compare_hyperparameters

In [ ]:
seeds = configs2.hparams_comparison_seeds
compare_hyperparameters.seeds = seeds
configuration = {
        'test_room': 2,
        'seed': tune.grid_search(seeds),
        'batch_size': 16,
        'epoch': 5,
        'data_dir': pathlib.Path('/data/wzy/WifiEnvironment/gram_data')}
hyper_paremeters = {
'slicing': {
        # 'slice_stride' : tune.grid_search([x // time_resolution for x in [100, 200, 300]]),
        'slice_length': tune.grid_search([x // configs2.time_resolution for x in [300, 400, 500, 600, 700, 800]])},
#'label_smoothening': {
#        'label_smoothening': tune.grid_search([0.0, 0.1, 0.2])},
#'smoothness_regularization': {
#        'lambda_smoothness': tune.grid_search([0.0, 0.1, 0.2, 0.3, 0.4])},
'gram_type': {
        'gram_type': tune.grid_search(['log_stft', 'log_hht', 'stft', 'hht', 'ampl'])},}
#'learning_rate': {
#        'learning_rate': tune.grid_search([1e-4, 3e-4, 5e-4, 7e-4, 1e-3, 2e-3]),
#        'learning_rate_disc': tune.grid_search([1e-4, 3e-4, 5e-4, 7e-4, 1e-3, 2e-3])}

Initialize ray

In [ ]:
ray.init()

In [ ]:
experiments = []
for key, value in hyper_paremeters.items():
    value.update(configuration)
    experiment = tune.Experiment(
            name = key,
            run = compare_hyperparameters.Trainable,
            stop = {'training_iteration': 20, },
            config = value,
            resources_per_trial = {
                    'cpu': 1,
                    'gpu': 1},
            num_samples = 1,
            local_dir = output_dir,
            checkpoint_freq = 0,
            checkpoint_at_end = True,
            max_failures = 5)
    experiments.append(experiment)

In [ ]:
trails = tune.run_experiments(
        experiments,
        verbose = 1,
        raise_on_failed_trial = False)

In [ ]:
ray.shutdown()